https://colab.research.google.com/drive/1xN2COoN8HJhy7ILmuI47zekw2EPNlIAm#scrollTo=kgSuc3rYKo6y
https://github.com/keras-team/keras/tree/master/examples

Connect your google drive 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load the dataset in data variable


In [2]:
# here we will import the libraries used for machine learning
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph.
from sklearn.linear_model import LogisticRegression # to apply the Logistic regression
from sklearn.model_selection import train_test_split # to split the data into two parts
# from sklearn.cross_validation import KFold # use for cross validation
from sklearn.model_selection import GridSearchCV# for tuning parameter
from sklearn.ensemble import RandomForestClassifier # for random forest classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm # for Support Vector Machine
from sklearn import metrics # for the check the error and accuracy of the model
# Any results you write to the current directory are saved as output.
# dont worry about the error if its not working then insteda of model_selection we can use cross_validation


In [3]:
# Get some time series data
data = pd.read_csv("/content/drive/My Drive/Google-CoLab/CourseML_2022_CAMSATS/11. Sample LSTM 001/timeseries-ds.csv",
                     header=0)# here header 0 means the 0 th row is our coloumn 

# have a look at the data
print(data.head())# as u can see our data have imported and having 33 columns

         Date      A       B       C      D      E      F      G
0  2008-03-18  24.68  164.93  114.73  26.27  19.21  28.87  63.44
1  2008-03-19  24.18  164.89  114.75  26.22  19.07  27.76  59.98
2  2008-03-20  23.99  164.63  115.04  25.78  19.01  27.04  59.61
3  2008-03-25  24.14  163.92  114.85  27.41  19.61  27.84  59.41
4  2008-03-26  24.44  163.45  114.84  26.86  19.53  28.02  60.09


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    11 non-null     object 
 1   A       11 non-null     float64
 2   B       11 non-null     float64
 3   C       11 non-null     float64
 4   D       11 non-null     float64
 5   E       11 non-null     float64
 6   F       11 non-null     float64
 7   G       11 non-null     float64
dtypes: float64(7), object(1)
memory usage: 832.0+ bytes


In [ ]:
# now we can drop this column Unnamed: 32
##data.drop("samples_below_80pct_ff",axis=0,inplace=True) # in this process this will change in our data itself 
# if you want to save your old data then you can use below code
# data1=data.drop("Unnamed:32",axis=1)
# here axis 1 means we are droping the column

# data.drop("samples_below_75pct_ff",axis=1,inplace=True)
# data.drop("samples_below_70pct_ff",axis=1,inplace=True)
# data.drop("samples_below_65pct_ff",axis=1,inplace=True)
# data.drop("samples_below_60pct_ff",axis=1,inplace=True)
# data.drop("samples_below_55pct_ff",axis=1,inplace=True)
# data.drop("samples_below_50pct_ff",axis=1,inplace=True)
# data.drop("samples_below_45pct_ff",axis=1,inplace=True)
# data.drop("samples_below_40pct_ff",axis=1,inplace=True)
# data.drop("samples_below_35pct_ff",axis=1,inplace=True)
# data.drop("samples_below_30pct_ff",axis=1,inplace=True)
# data.drop("samples_below_25pct_ff",axis=1,inplace=True)
# data.drop("samples_below_20pct_ff",axis=1,inplace=True)
# data.drop("samples_below_15pct_ff",axis=1,inplace=True)
# data.drop("samples_below_10pct_ff",axis=1,inplace=True)
# data.drop("samples_below_5pct_ff",axis=1,inplace=True)

# here you can check the column has been droped
data.columns # this gives the column name which are persent in our data no Unnamed: 32 is not now there

Index(['Date', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='object')

In [5]:

print(data.shape)
print("-----------------------------------")
data.describe() # this will describe the all statistical function of our data

(11, 8)
-----------------------------------


,A,B,C,D,E,F,G
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000
mean,24.227273,163.888182,115.155455,27.061818,19.665455,28.410000,59.119091
std,0.239879,0.647438,0.352686,0.795510,0.487737,0.759697,1.964568
min,23.810000,163.220000,114.730000,25.780000,19.010000,27.040000,56.180000
25%,24.085000,163.395000,114.845000,26.505000,19.370000,27.930000,58.070000
50%,24.190000,163.590000,115.110000,27.090000,19.610000,28.250000,59.410000
75%,24.360000,164.325000,115.470000,27.615000,19.965000,29.020000,59.800000
max,24.680000,164.930000,115.720000,28.220000,20.420000,29.510000,63.440000


You can build put inputs into a vector and then use pandas .cumsum() function to build the sequence for the time series:


In [6]:
#input_cols = ['Date', 'A', 'B', 'C', 'D', 'E', 'F']
input_cols = ['A', 'B', 'C', 'D', 'E', 'F']

# Put your inputs into a single list
data['single_input_vector'] = data[input_cols].apply(tuple, axis=1).apply(list)
# Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
data['single_input_vector'] = data.single_input_vector.apply(lambda x: [list(x)])
# Use .cumsum() to include previous row vectors in the current row list of vectors
data['cumulative_input_vectors'] = data.single_input_vector.cumsum()

The output can be set up in a similar way, but it will be a single vector instead of a sequence:

In [7]:
output_cols = ['G']
# If your output is multi-dimensional, you need to capture those dimensions in one object
# If your output is a single dimension, this step may be unnecessary
data['output_vector'] = data[output_cols].apply(tuple, axis=1).apply(list)

The input sequences have to be the same length to run them through the model, so you need to pad them to be the max length of your cumulative vectors:

In [8]:
# Pad your sequences so they are the same length
from keras.preprocessing.sequence import pad_sequences
from pandas import Series, DataFrame

max_sequence_length = data.cumulative_input_vectors.apply(len).max()
# Save it as a list   
padded_sequences = pad_sequences(data.cumulative_input_vectors.tolist(), max_sequence_length).tolist()
data['padded_input_vectors'] = pd.Series(padded_sequences).apply(np.asarray)

Training data can be pulled from the dataframe and put into numpy arrays. Note that the input data that comes out of the dataframe will not make a 3D array. It makes an array of arrays, which is not the same thing.

You can use hstack and reshape to build a 3D input array.

In [9]:
# Extract your training data
X_train_init = np.asarray(data.padded_input_vectors)
# Use hstack to and reshape to make the inputs a 3d vector
X_train = np.hstack(X_train_init).reshape(len(data),max_sequence_length,len(input_cols))
y_train = np.hstack(np.asarray(data.output_vector)).reshape(len(data),len(output_cols))

Testing so far

In [10]:
print(X_train_init.shape)
print(X_train.shape)
print(X_train == X_train_init)

(11,)
(11, 11, 6)
False


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


Once you have training data you can define the dimensions of your input layer and output layers.

In [11]:
# Get your input dimensions
# Input length is the length for one input sequence (i.e. the number of rows for your sample)
# Input dim is the number of dimensions in one input vector (i.e. number of input columns)
input_length = X_train.shape[1]
input_dim = X_train.shape[2]
# Output dimensions is the shape of a single output vector
# In this case it's just 1, but it could be more
output_dim = len(y_train[0])

Build the model:

In [12]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense

# Build the model
model = Sequential()

# I arbitrarily picked the output dimensions as 4
model.add(LSTM(4, input_dim = input_dim, input_length = input_length))
# The max output value is > 1 so relu is used as final activation.
model.add(Dense(output_dim, activation='relu'))

model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
# Set batch_size to 7 to show that it doesn't have to be a factor or multiple of your sample size
history = model.fit(X_train, y_train,
              batch_size=7, nb_epoch=50,
              verbose = 1)

Epoch 1/50
11/11 [==============================] - 0s 2ms/step - loss: 52.7813 - acc: 0.0000e+00
Epoch 2/50
11/11 [==============================] - 0s 2ms/step - loss: 52.2858 - acc: 0.0000e+00
Epoch 3/50
11/11 [==============================] - 0s 1ms/step - loss: 52.0275 - acc: 0.0000e+00
Epoch 4/50
11/11 [==============================] - 0s 2ms/step - loss: 51.7764 - acc: 0.0000e+00
Epoch 5/50
11/11 [==============================] - 0s 1ms/step - loss: 51.5528 - acc: 0.0000e+00
Epoch 6/50
11/11 [==============================] - 0s 1ms/step - loss: 51.4040 - acc: 0.0000e+00
Epoch 7/50
11/11 [==============================] - 0s 1ms/step - loss: 50.8425 - acc: 0.0000e+00
Epoch 8/50
11/11 [==============================] - 0s 1ms/step - loss: 50.6567 - acc: 0.0000e+00
Epoch 9/50
11/11 [==============================] - 0s 1ms/step - loss: 50.3904 - acc: 0.0000e+00
Epoch 10/50
11/11 [==============================] - 0s 2ms/step - loss: 50.1509 - acc: 0.0000e+00
Epoch 11/50
11/11 [

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 12/50
11/11 [==============================] - 0s 1ms/step - loss: 49.4448 - acc: 0.0000e+00
Epoch 13/50
11/11 [==============================] - 0s 1ms/step - loss: 49.2546 - acc: 0.0000e+00
Epoch 14/50
11/11 [==============================] - 0s 1ms/step - loss: 48.9598 - acc: 0.0000e+00
Epoch 15/50
11/11 [==============================] - 0s 1ms/step - loss: 48.7100 - acc: 0.0000e+00
Epoch 16/50
11/11 [==============================] - 0s 2ms/step - loss: 48.6208 - acc: 0.0000e+00
Epoch 17/50
11/11 [==============================] - 0s 1ms/step - loss: 48.3081 - acc: 0.0000e+00
Epoch 18/50
11/11 [==============================] - 0s 2ms/step - loss: 48.0894 - acc: 0.0000e+00
Epoch 19/50
11/11 [==============================] - 0s 1ms/step - loss: 47.9621 - acc: 0.0000e+00
Epoch 20/50
11/11 [==============================] - 0s 1ms/step - loss: 47.3647 - acc: 0.0000e+00
Epoch 21/50
11/11 [==============================] - 0s 1ms/step - loss: 47.1807 - acc: 0.0000e+00
Epoch 22/5

Use model.predict(X) where X is the same format (other than the number of samples) as X_train in order to make predictions from the model.

In [13]:
#prediction code in here

In [14]:
from datetime import datetime

graph_xy_var = 'utc_time_id'
# now these are the variables which will use for prediction
X_graph_data = np.array(train[graph_xy_var])     # taking the training data input 

arr = []
list = X_graph_data;
index=0 #Start at the first element
while(index < len(list)):   # Until we get to the end of the list
    KK = (pd.to_datetime(list[index]) - pd.to_datetime(list[0])).total_seconds()
    # KK = (pd.to_datetime(list[0]) - pd.to_datetime(list[index])).dt.total_seconds()
    arr.append( KK );
    index+=1 #Move The first element n elements forward

print(X_graph_data);
print(arr)
 
Y_graph_data = np.array(train['avg_freeflow_speed'], )
print(Y_graph_data);
 
import altair as alt
alt.Chart(train).mark_line().encode(
  x='utc_time_id:T',
  y='avg_travel_time',
  color='avg_freeflow_speed'
).interactive(bind_y=False)

NameError: ignored

In [15]:
# Graph try
prediction_var = [
        'utc_time_id', 
        'avg_freeflow_speed', 
        'avg_travel_time',
       'high_quality_samples', 'samples_below_100pct_ff', 'samples_below_95pct_ff', 'samples_below_90pct_ff',
       'samples_below_85pct_ff', 'samples_below_80pct_ff', 'samples_below_75pct_ff', 'samples_below_70pct_ff',
       'samples_below_65pct_ff', 'samples_below_60pct_ff', 'samples_below_55pct_ff', 'samples_below_50pct_ff',
       'samples_below_45pct_ff', 'samples_below_40pct_ff', 'samples_below_35pct_ff', 'samples_below_30pct_ff',
       'samples_below_25pct_ff', 'samples_below_20pct_ff', 'samples_below_15pct_ff', 'samples_below_10pct_ff',
       'samples_below_5pct_ff']

Now I'm going to use Keras to define a Neural network that will be trained off of this data. This Neural Network can then be used to predict future values for breast cancer presence. 

In [16]:
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential() # Initialising the ANN

classifier.add(Dense(units = 100, activation = 'relu', input_dim = 25))
classifier.add(Dense(units = 50, activation = 'tanh'))
classifier.add(Dense(units = 20, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

Once the Neural Network is defined, I now have to specify is the optimizer and loss function

In [17]:
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')

You now train the neural network using Classifier.fit, passing it the training data -- i.e. for this set of X, this is what the Y should look like. The NN will then spot the patterns in the data, and build a neural network that could replicate that. 

In [18]:
classifier.fit(X_train, Y_train, batch_size = 1, epochs = 10)

NameError: ignored

To predict new values, the Neural Network uses classifier.predict. I'm going to pass it the test values for X (which the Neural Network hasn't previously seen) and it will give me back a set of predictions. These predicitons will be probabilities, so I will clean them up by saying that if thye are greater than .5, I'll make them 1, else I'll make them 0.

In [19]:
Y_pred = classifier.predict(X_test)
# print(Y_pred);
Y_pred = [ 1 if y>=0.5 else 0 for y in Y_pred ]

print(Y_pred);

NameError: ignored

Now we can loop through the set of predicitons for the test set (called Y_pred) and the actual values for the test set (celled Y_test), and see how alike they are -- if they are the same, I'll increment 'correct', otherwise I'll incrememnt 'wrong'. 

You'll see the result is 100% accurate, even though the neural network reported a lower accuracy than that. Why? 

In [20]:
total = 0
correct = 0
wrong = 0
print(Y_pred[9]);
for i in Y_pred:
  total=total+1
  
  if(Y_test.at[i,0] == Y_pred[i]):
    correct=correct+1
  else:
    wrong=wrong+1

print("Total " + str(total))
print("Correct " + str(correct))
print("Wrong " + str(wrong))

NameError: ignored